In [ ]:
import requests
import sys
import time
import base64

# Fill in Collibra Connection info
COLLIBRA_API_URL = 'https://<instanceURL>/rest/2.0'
USERNAME = '<your username>'
# base64 encode/decode password
PASSWORD = base64.b64decode('<yourEncryptedPassword>'.encode('utf-8')).decode('utf-8')

# Collibar JSON Import file to import
IMPORT_JSON_FILE = '<specify the location of your Update File>'

# Duration in seconds before polling job status again.
WAIT_DURATION = 2

In [ ]:
# Import the metadata into Collibra
# If there is a hand-shake error after running this cell, comment out the files=files line and use the last lines instead
# Then rerun all the cells in this application
files = {'file': open(IMPORT_JSON_FILE, 'rb')}

print('********************* Starting Import *********************')

response = requests.post(
        COLLIBRA_API_URL + '/import/json-job', 
        auth=(USERNAME, PASSWORD),
        files=files
        #files=files, verify = False
    )
response.text

In [ ]:
# Read the returned job ID and state
jobId = response.json()['id']
state = response.json()['state']


In [ ]:
# Wait for the import job to finish
# If there is a hand-shake error after running this cell, comment out the auth=USERNAME, PASSWORD line and use the last lines instead
# Then rerun all the cells in this application
if state == 'WAITING':
    
    print('Waiting for import job to finish: ' + jobId)
    
    while state != 'COMPLETED' and state !='ERROR':
        response = requests.get(
            COLLIBRA_API_URL + '/jobs/' + jobId, 
            auth=(USERNAME, PASSWORD)
            #auth=(USERNAME, PASSWORD), verify = False
        )
        state = response.json()['state']
        print('Current job state: ' + state)
        time.sleep(WAIT_DURATION)
   
    print('Finished import job: ' + jobId)
    print('********************* ALL DONE *********************')
        

else:
    raise ValueError('Collibra import job did not start correctly.')
